In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim
import requests # library to handle requests
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
!conda install -c conda-forge folium=0.5.0 --yes
import folium
print('Libraries imported.')

Solving environment: ...working... 
  - anaconda/win-64::ca-certificates-2020.1.1-0, anaconda/win-64::openssl-1.1.1d-he774522_4
  - anaconda/win-64::ca-certificates-2020.1.1-0, defaults/win-64::openssl-1.1.1d-he774522_4
  - anaconda/win-64::openssl-1.1.1d-he774522_4, defaults/win-64::ca-certificates-2020.1.1-0
  - defaults/win-64::ca-certificates-2020.1.1-0, defaults/win-64::openssl-1.1.1d-he774522_4done

# All requested packages already installed.

Solving environment: ...working... 
  - anaconda/win-64::ca-certificates-2020.1.1-0, anaconda/win-64::openssl-1.1.1d-he774522_4
  - anaconda/win-64::openssl-1.1.1d-he774522_4, defaults/win-64::ca-certificates-2020.1.1-0
  - anaconda/win-64::ca-certificates-2020.1.1-0, defaults/win-64::openssl-1.1.1d-he774522_4
  - defaults/win-64::ca-certificates-2020.1.1-0, defaults/win-64::openssl-1.1.1d-he774522_4done

# All requested packages already installed.

Libraries imported.


In [2]:
dstr=pd.read_csv('Boroughs.csv')

In [3]:
dstr

,Borough,Lat,Long
0,Charlottenburg-Wilmersdorf,52.500000,13.283333
1,Friedrichshain-Kreuzberg,52.500000,13.450000
2,Lichtenberg,52.533333,13.500000
3,Marzahn-Hellersdorf,52.533333,13.583333
4,Mitte,52.516667,13.366667
5,Neukölln,52.483333,13.450000
6,Pankow,52.566667,13.400000
7,Reinickendorf,52.566667,13.333333
8,Spandau,52.550000,13.200000
9,Steglitz-Zehlendorf,52.433333,13.250000


In [4]:
map_berlin = folium.Map(location=[52.5200, 13.4050], zoom_start=10)

for borough, lat, lng in zip(dstr['Borough'],dstr['Lat'], dstr['Long']):
    label = '{}'.format(borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_berlin) 

map_berlin

In [50]:
berlincov=pd.read_html('https://www.citypopulation.de/en/germany/covid/11__berlin/')

In [51]:
berlincov=berlincov[0]

In [96]:
berlincov.rename(columns={'Name': 'Borough'}, inplace=True)

In [95]:
berlincov

,Name,Total
1,Charlottenburg-Wilmersdorf,627
2,Friedrichshain-Kreuzberg,434
3,Lichtenberg,202
4,Marzahn-Hellersdorf,238
5,Mitte,806
6,Neukölln,568
7,Pankow,552
8,Reinickendorf,423
9,Spandau,218
10,Steglitz-Zehlendorf,443


In [97]:
Berlin = pd.merge(dstr, berlincov, on=['Borough'], how='left')

In [98]:
Berlin

,Borough,Lat,Long,Total
0,Charlottenburg-Wilmersdorf,52.500000,13.283333,627
1,Friedrichshain-Kreuzberg,52.500000,13.450000,434
2,Lichtenberg,52.533333,13.500000,202
3,Marzahn-Hellersdorf,52.533333,13.583333,238
4,Mitte,52.516667,13.366667,806
5,Neukölln,52.483333,13.450000,568
6,Pankow,52.566667,13.400000,552
7,Reinickendorf,52.566667,13.333333,423
8,Spandau,52.550000,13.200000,218
9,Steglitz-Zehlendorf,52.433333,13.250000,443


In [99]:

# create map of Toronto using latitude and longitude values
d = folium.Map(location=[52.5200, 13.4050], zoom_start=11)

# add markers to map
for borough, lat, lng, case in zip(dstr['Borough'],dstr['Lat'], dstr['Long'], Berlin['Total']):
    label = '{},{}'.format(borough,case)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=case*0.055,
        popup=label,
        color='black',
        fill=True,
        fill_color='#FF756B',
        fill_opacity=0.7,
        parse_html=False).add_to(d) 

d

In [101]:
CLIENT_ID = 'PYJJZTQEN0YEJ1EQ3TCECTCAADQE0HWE2ISUVVCTAPQVFR0V' # your Foursquare ID
CLIENT_SECRET = '13O4X2JV50MKERH0NIZFRU032OQWJHCVZEGQ1C1OHDN2NZYK' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: PYJJZTQEN0YEJ1EQ3TCECTCAADQE0HWE2ISUVVCTAPQVFR0V
CLIENT_SECRET:13O4X2JV50MKERH0NIZFRU032OQWJHCVZEGQ1C1OHDN2NZYK


In [119]:
LIMIT = 200 # limit of number of venues returned by Foursquare API

radius = 2000 # define radius

In [120]:
def getNearbyVenues(names, latitudes, longitudes, radius=2000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Borough', 
                  'Borough Latitude', 
                  'Borough Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [121]:
Berlin_venues = getNearbyVenues(names=Berlin['Borough'],
                                   latitudes=Berlin['Lat'],
                                   longitudes=Berlin['Long']
                                  )

Charlottenburg-Wilmersdorf
Friedrichshain-Kreuzberg
Lichtenberg
Marzahn-Hellersdorf
Mitte
Neukölln
Pankow
Reinickendorf
Spandau
Steglitz-Zehlendorf
Tempelhof-Schöneberg
Treptow-Köpenick


In [156]:
print(Berlin_venues.shape)
Berlin_venues.head()

(1076, 7)


,Borough,Borough Latitude,Borough Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Charlottenburg-Wilmersdorf,52.5,13.283333,Lietzenseepark,52.504977,13.287041,Park
1,Charlottenburg-Wilmersdorf,52.5,13.283333,Cups,52.497388,13.291307,Café
2,Charlottenburg-Wilmersdorf,52.5,13.283333,Fleischerei Bünger,52.496390,13.292842,Butcher
3,Charlottenburg-Wilmersdorf,52.5,13.283333,Scoopy doo,52.498363,13.290624,Ice Cream Shop
4,Charlottenburg-Wilmersdorf,52.5,13.283333,Fressnapf,52.499663,13.288616,Pet Store


In [123]:
Berlin_venues.groupby('Borough').count()

,Borough Latitude,Borough Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Borough,,,,,,
Charlottenburg-Wilmersdorf,100,100,100,100,100,100
Friedrichshain-Kreuzberg,100,100,100,100,100,100
Lichtenberg,91,91,91,91,91,91
Marzahn-Hellersdorf,72,72,72,72,72,72
Mitte,100,100,100,100,100,100
Neukölln,100,100,100,100,100,100
Pankow,100,100,100,100,100,100
Reinickendorf,68,68,68,68,68,68
Spandau,91,91,91,91,91,91


In [124]:
print('There are {} uniques categories.'.format(len(Berlin_venues['Venue Category'].unique())))

There are 222 uniques categories.


In [139]:
# one hot encoding
Berlin_onehot = pd.get_dummies(Berlin_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Berlin_onehot['Borough'] = Berlin_venues['Borough'] 

# move neighborhood column to the first column
fixed_columns = [Berlin_onehot.columns[-1]] + list(Berlin_onehot.columns[:-1])
Berlin_onehot = Berlin_onehot[fixed_columns]

Berlin_onehot.head()

,Borough,ATM,African Restaurant,American Restaurant,Amphitheater,Argentinian Restaurant,Art Gallery,Art Museum,Asian Restaurant,Athletics & Sports,Austrian Restaurant,Auto Dealership,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Bathing Area,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Shop,Bistro,Bookstore,Bowling Alley,Breakfast Spot,Brewery,Bridge,Building,Burger Joint,Bus Stop,Business Service,Butcher,Cable Car,Cafeteria,Café,Camera Store,Candy Store,Capitol Building,Castle,Caucasian Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Concert Hall,Construction & Landscaping,Convenience Store,Cycle Studio,Deli / Bodega,Department Store,Dessert Shop,Distillery,Dive Bar,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Event Service,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Flower Shop,Food & Drink Shop,Food Court,Food Service,Forest,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,German Restaurant,Gift Shop,Go Kart Track,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Hardware Store,Historic Site,History Museum,Hockey Rink,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indoor Play Area,Irish Pub,Italian Restaurant,Japanese Restaurant,Korean Restaurant,Lake,Lebanese Restaurant,Light Rail Station,Liquor Store,Lounge,Market,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Motorcycle Shop,Mountain,Movie Theater,Multiplex,Museum,Music Venue,Neighborhood,Newsstand,Nightclub,Noodle House,Optical Shop,Organic Grocery,Outdoor Sculpture,Paintball Field,Pakistani Restaurant,Palace,Palatine Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pide Place,Pie Shop,Pizza Place,Playground,Plaza,Pool,Pool Hall,Pub,Record Shop,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Rest Area,Restaurant,Rock Climbing Spot,Rock Club,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Snack Place,Soccer Field,Soccer Stadium,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Club,Squash Court,Stadium,Stationery Store,Steakhouse,Supermarket,Sushi Restaurant,Tapas Restaurant,Taverna,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park Ride / Attraction,Theme Restaurant,Toy / Game Store,Trail,Train Station,Tram Station,Trattoria/Osteria,Tree,Tunnel,Turkish Restaurant,Vacation Rental,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Volleyball Court,Waterfront,Whisky Bar,Windmill,Wine Bar,Wine Shop,Women's Store,Yoga Studio,Zoo Exhibit
0,Charlottenburg-Wilmersdorf,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Charlottenburg-Wilmersdorf,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [140]:
Berlin_onehot.shape

(1076, 223)

In [141]:
Berlin_grouped = Berlin_onehot.groupby('Borough').mean().reset_index()
Berlin_grouped

,Borough,ATM,African Restaurant,American Restaurant,Amphitheater,Argentinian Restaurant,Art Gallery,Art Museum,Asian Restaurant,Athletics & Sports,Austrian Restaurant,Auto Dealership,Automotive Shop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Bathing Area,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Shop,Bistro,Bookstore,Bowling Alley,Breakfast Spot,Brewery,Bridge,Building,Burger Joint,Bus Stop,Business Service,Butcher,Cable Car,Cafeteria,Café,Camera Store,Candy Store,Capitol Building,Castle,Caucasian Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Concert Hall,Construction & Landscaping,Convenience Store,Cycle Studio,Deli / Bodega,Department Store,Dessert Shop,Distillery,Dive Bar,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Event Service,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Flower Shop,Food & Drink Shop,Food Court,Food Service,Forest,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,German Restaurant,Gift Shop,Go Kart Track,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Hardware Store,Historic Site,History Museum,Hockey Rink,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indoor Play Area,Irish Pub,Italian Restaurant,Japanese Restaurant,Korean Restaurant,Lake,Lebanese Restaurant,Light Rail Station,Liquor Store,Lounge,Market,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Motorcycle Shop,Mountain,Movie Theater,Multiplex,Museum,Music Venue,Neighborhood,Newsstand,Nightclub,Noodle House,Optical Shop,Organic Grocery,Outdoor Sculpture,Paintball Field,Pakistani Restaurant,Palace,Palatine Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pide Place,Pie Shop,Pizza Place,Playground,Plaza,Pool,Pool Hall,Pub,Record Shop,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Rest Area,Restaurant,Rock Climbing Spot,Rock Club,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Snack Place,Soccer Field,Soccer Stadium,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Club,Squash Court,Stadium,Stationery Store,Steakhouse,Supermarket,Sushi Restaurant,Tapas Restaurant,Taverna,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park Ride / Attraction,Theme Restaurant,Toy / Game Store,Trail,Train Station,Tram Station,Trattoria/Osteria,Tree,Tunnel,Turkish Restaurant,Vacation Rental,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Volleyball Court,Waterfront,Whisky Bar,Windmill,Wine Bar,Wine Shop,Women's Store,Yoga Studio,Zoo Exhibit
0,Charlottenburg-Wilmersdorf,0.000000,0.00,0.01,0.000000,0.000000,0.00,0.000000,0.030000,0.000000,0.00,0.000000,0.000000,0.00,0.00,0.010000,0.030000,0.000000,0.020000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.010000,0.010000,0.00,0.000000,0.00,0.00,0.020000,0.000000,0.00,0.01,0.000000,0.000000,0.080000,0.00,0.00,0.00,0.000000,0.00,0.00,0.010000,0.000000,0.00,0.000000,0.010000,0.01,0.01,0.01,0.000000,0.000000,0.00,0.00,0.000000,0.01,0.00,0.00,0.020000,0.000000,0.020000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.00,0.01,0.00,0.000000,0.000000,0.010000,0.000000,0.000000,0.010000,0.000000,0.000000,0.000000,0.040000,0.00,0.000000,0.010000,0.010000,0.010000,0.000000,0.030000,0.00,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.050000,0.00,0.040000,0.010000,0.00,0.00,0.000000,0.00,0.100000,0.01,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.

In [142]:
Berlin_grouped.shape

(12, 223)

In [143]:
num_top_venues = 5

for hood in Berlin_grouped['Borough']:
    print("----"+hood+"----")
    temp = Berlin_grouped[Berlin_grouped['Borough'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Charlottenburg-Wilmersdorf----
                venue  freq
0  Italian Restaurant  0.10
1                Café  0.08
2   Trattoria/Osteria  0.06
3               Hotel  0.05
4   German Restaurant  0.04


----Friedrichshain-Kreuzberg----
                venue  freq
0                Café  0.08
1      Ice Cream Shop  0.07
2         Coffee Shop  0.05
3           Nightclub  0.04
4  Falafel Restaurant  0.04


----Lichtenberg----
                    venue  freq
0             Supermarket  0.11
1            Tram Station  0.07
2                    Park  0.04
3  Furniture / Home Store  0.04
4               Drugstore  0.03


----Marzahn-Hellersdorf----
           venue  freq
0    Supermarket  0.17
1         Garden  0.11
2   Tram Station  0.08
3      Drugstore  0.07
4  Shopping Mall  0.04


----Mitte----
                 venue  freq
0                Hotel  0.10
1         Concert Hall  0.06
2                Plaza  0.05
3          Coffee Shop  0.04
4  Monument / Landmark  0.04


----Neukölln----
   

In [157]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [168]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Borough']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Borough'] = Berlin_grouped['Borough']

for ind in np.arange(Berlin_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Berlin_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head(10)

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Charlottenburg-Wilmersdorf,Italian Restaurant,Café,Trattoria/Osteria,Hotel,Organic Grocery,German Restaurant,Ice Cream Shop,Gym / Fitness Center,Bakery,Vietnamese Restaurant
1,Friedrichshain-Kreuzberg,Café,Ice Cream Shop,Coffee Shop,Bar,Nightclub,Falafel Restaurant,Vietnamese Restaurant,Brewery,Middle Eastern Restaurant,Pizza Place
2,Lichtenberg,Supermarket,Tram Station,Park,Furniture / Home Store,Drugstore,Café,Gym / Fitness Center,Vietnamese Restaurant,Bakery,Rental Car Location
3,Marzahn-Hellersdorf,Supermarket,Garden,Tram Station,Drugstore,Shopping Mall,Ice Cream Shop,Bus Stop,Italian Restaurant,Gas Station,ATM
4,Mitte,Hotel,Concert Hall,Plaza,Science Museum,Monument / Landmark,Coffee Shop,Spa,Art Museum,Shopping Mall,Chocolate Shop
5,Neukölln,Coffee Shop,Café,Bar,Vegetarian / Vegan Restaurant,Breakfast Spot,Bakery,Park,Bistro,Plaza,Nightclub
6,Pankow,Café,Supermarket,Italian Restaurant,Drugstore,Park,Bar,Gym / Fitness Center,Ice Cream Shop,Coffee Shop,Pub
7,Reinickendorf,Supermarket,Park,Bakery,Metro Station,Restaurant,Big Box Store,Bar,Plaza,Gas Station,Electronics Store
8,Spandau,Supermarket,Bus Stop,Drugstore,German Restaurant,Clothing Store,Italian Restaurant,Bakery,Fast Food Restaurant,Park,Trattoria/Osteria
9,Steglitz-Zehlendorf,Italian Restaurant,Café,Bus Stop,Supermarket,Doner Restaurant,German Restaurant,Steakhouse,Drugstore,Light Rail Station,Gym / Fitness Center


In [169]:
kclusters = 3

Berlin_grouped_clustering = Berlin_grouped.drop('Borough', 1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Berlin_grouped_clustering)

kmeans.labels_[0:10] 

array([2, 0, 1, 1, 0, 0, 2, 1, 1, 2])

In [170]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Berlin_merged = Berlin

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Berlin_merged = Berlin_merged.join(neighborhoods_venues_sorted.set_index('Borough'), on='Borough')


Berlin_merged.head(12) # check the last columns!

,Borough,Lat,Long,Total,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Charlottenburg-Wilmersdorf,52.500000,13.283333,627,2,Italian Restaurant,Café,Trattoria/Osteria,Hotel,Organic Grocery,German Restaurant,Ice Cream Shop,Gym / Fitness Center,Bakery,Vietnamese Restaurant
1,Friedrichshain-Kreuzberg,52.500000,13.450000,434,0,Café,Ice Cream Shop,Coffee Shop,Bar,Nightclub,Falafel Restaurant,Vietnamese Restaurant,Brewery,Middle Eastern Restaurant,Pizza Place
2,Lichtenberg,52.533333,13.500000,202,1,Supermarket,Tram Station,Park,Furniture / Home Store,Drugstore,Café,Gym / Fitness Center,Vietnamese Restaurant,Bakery,Rental Car Location
3,Marzahn-Hellersdorf,52.533333,13.583333,238,1,Supermarket,Garden,Tram Station,Drugstore,Shopping Mall,Ice Cream Shop,Bus Stop,Italian Restaurant,Gas Station,ATM
4,Mitte,52.516667,13.366667,806,0,Hotel,Concert Hall,Plaza,Science Museum,Monument / Landmark,Coffee Shop,Spa,Art Museum,Shopping Mall,Chocolate Shop
5,Neukölln,52.483333,13.450000,568,0,Coffee Shop,Café,Bar,Vegetarian / Vegan Restaurant,Breakfast Spot,Bakery,Park,Bistro,Plaza,Nightclub
6,Pankow,52.566667,13.400000,552,2,Café,Supermarket,Italian Restaurant,Drugstore,Park,Bar,Gym / Fitness Center,Ice Cream Shop,Coffee Shop,Pub
7,Reinickendorf,52.566667,13.333333,423,1,Supermarket,Park,Bakery,Metro Station,Restaurant,Big Box Store,Bar,Plaza,Gas Station,Electronics Store
8,Spandau,52.550000,13.200000,218,1,Supermarket,Bus Stop,Drugstore,German Restaurant,Clothing Store,Italian Restaurant,Bakery,Fast Food Restaurant,Park,Trattoria/Osteria
9,Steglitz-Zehlendorf,52.433333,13.250000,443,2,Italian Restaurant,Café,Bus Stop,Supermarket,Doner Restaurant,German Restaurant,Steakhouse,Drugstore,Light Rail Station,Gym / Fitness Center


In [171]:
# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Berlin_merged['Lat'], Berlin_merged['Long'], Berlin_merged['Borough'], Berlin_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(d)
       
d